In [1]:
import pandas as pd

In [15]:
import requests
import json
import pandas as pd
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import re
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook

disable_warnings(InsecureRequestWarning)

base_url = "https://viperdb.scripps.edu"

all_tnums_url = "/services/tnumber_index.php?serviceName=tnumbers"
all_genuses_url = "/services/genus_index.php?serviceName=genus"
all_families_url = "/services/family_index.php?serviceName=families"

tnum_members_url = "/services/tnumber_index.php?serviceName=tnumber_members&tnumber="
genus_members_url = "/services/genus_index.php?serviceName=genus_members&genus="
family_members_url = "/services/family_index.php?serviceName=family_members&family="

rcsb_url = 'https://data.rcsb.org/rest/v1/core/entry/'
vdb_url = "https://viperdb.scripps.edu/"
radius_url = "services/biodata.php?serviceName=radius_formatted&VDB="

def getrcsb(c):
    res = requests.get(rcsb_url + c, verify=False)
    return res
def getrad(c):
    res = requests.get(vdb_url + radius_url + c, verify=False)
    return res

def tnum_url(n):
    return base_url + tnum_members_url + str(n)
def genus_url(n):
    return base_url + genus_members_url + str(n)
def family_url(n):
    return base_url + family_members_url + str(n)

#sort list of dictionaries by resolution and take the n lowest
def lowres_dictlist(dictlist, n):
    lowres = {}
    for k in dictlist.keys():
        sortres = sorted(dictlist[k], key=lambda d: (d['resolution'] is None, d['resolution']))
        lowres[k] = sortres[:n]
    return lowres
def dictlist(dictlist):
    res = {}
    for k in dictlist.keys():
        sortres = sorted(dictlist[k], key=lambda d: (d['resolution'] is None, d['resolution']))
        res[k] = sortres
    return res
families_res = requests.get((base_url + all_families_url), verify=False)
families = json.loads(families_res.text)

In [2]:
file_url = 'families.xlsx'
fam_xl = pd.ExcelFile(file_url)

In [17]:
#get families containing each FSF
file_url = 't2.txt'
fsf_families = {}
with open(file_url, 'r') as f:
    lines = f.readlines()
    for l in lines:
        l = l.split()
        fams = [x.replace(',', '') for x in l[1:]]
        fsf_families[l[0]] = fams 
folds = list(fsf_families.keys())

In [29]:
non_tone = {}
dfs = []
for k, v in fsf_families.items():
    df_dict = {}
    for f in v:
        try:
            sheet = fam_xl.parse(f)
            tnums = sheet['tnumber']
            pdb_ids = sheet['id']
            nont1_ids = [pdb_ids[i] for i in list(range(len(tnums))) if tnums[i] != '1']
            df_dict[f] = nont1_ids
        except Exception as e:
            print(e)
    try:
        df = pd.DataFrame.from_dict(df_dict, orient='index')
        df = df.transpose()
        dfs.append(df)
    except Exception as e:
            print(e)

Worksheet named 'Orthomyxoviridae' not found
Worksheet named 'Coronaviridae' not found
Worksheet named 'Caudovirales' not found
Worksheet named 'Ophioviridae' not found
Worksheet named 'Poleomaviridae' not found
Worksheet named 'Ascoviridae' not found
Worksheet named 'Asfarviridae' not found
Worksheet named 'Iridoviridae' not found
Worksheet named 'Marsielleviridae' not found
Worksheet named 'Mimiviridae' not found
Worksheet named 'Inoviridae' not found
Worksheet named 'Inoviridae' not found
Worksheet named 'Arteriviridae' not found
Worksheet named 'Coronaviridae' not found
Worksheet named 'Benyviridae' not found
Worksheet named 'Potyviridae' not found
Worksheet named 'Virgaviridae' not found


In [30]:
len(dfs)

23

In [31]:
xl_file = 'non_t_one.xlsx'
wb = Workbook()
ws =  wb.active
wb.save(filename = xl_file)

book = load_workbook(xl_file)
writer = pd.ExcelWriter(xl_file, engine = 'openpyxl')
writer.book = book
for i, df in enumerate(dfs):
    fold = folds[i]
    
    df.to_excel(writer, sheet_name = fold, index = False)
del book['Sheet']
book.save(xl_file)